In [ ]:
# ! pip install h2o

In [3]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from __future__ import print_function
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_333"; Java(TM) SE Runtime Environment (build 1.8.0_333-b02); Java HotSpot(TM) 64-Bit Server VM (build 25.333-b02, mixed mode)
  Starting server from /Users/milou/opt/anaconda3/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/4w/s6bkpr6168z3_st_jrhjy5rh0000gn/T/tmpa9ps722g
  JVM stdout: /var/folders/4w/s6bkpr6168z3_st_jrhjy5rh0000gn/T/tmpa9ps722g/h2o_milou_started_from_python.out
  JVM stderr: /var/folders/4w/s6bkpr6168z3_st_jrhjy5rh0000gn/T/tmpa9ps722g/h2o_milou_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,3 days
H2O_cluster_name:,H2O_from_python_milou_xc3wrj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,14.20 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [4]:

# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:

# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)


In [6]:

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()


In [7]:

# Number of CV folds (to generate level-one data for stacking)
nfolds = 5



There are a few ways to assemble a list of models to stack together:

1. Train individual models and put them in a list
2. Train a grid of models
3. Train several grids of models

Note: All base models must have the same cross-validation folds and the cross-validated predicted values must be kept.




# 1. Generate a 2-model ensemble (GBM + RF)


In [8]:

# Train and cross-validate a GBM
my_gbm = H2OGradientBoostingEstimator(distribution="bernoulli",
                                      ntrees=10,
                                      max_depth=3,
                                      min_rows=2,
                                      learn_rate=0.2,
                                      nfolds=nfolds,
                                      fold_assignment="Modulo",
                                      keep_cross_validation_predictions=True,
                                      seed=1)
my_gbm.train(x=x, y=y, training_frame=train)


# Train and cross-validate a RF
my_rf = H2ORandomForestEstimator(ntrees=50,
                                 nfolds=nfolds,
                                 fold_assignment="Modulo",
                                 keep_cross_validation_predictions=True,
                                 seed=1)
my_rf.train(x=x, y=y, training_frame=train)


# Train a stacked ensemble using the GBM and GLM above
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_binomial",
                                       base_models=[my_gbm, my_rf])
ensemble.train(x=x, y=y, training_frame=train)

# Eval ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)

# Compare to base learner performance on the test set
perf_gbm_test = my_gbm.model_performance(test)
perf_rf_test = my_rf.model_performance(test)
baselearner_best_auc_test = max(perf_gbm_test.auc(), perf_rf_test.auc())
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))


gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
Best Base-learner Test AUC:  0.769204725074508
Ensemble Test AUC:  0.7731183158978566


In [9]:

# Generate predictions on a test set (if neccessary)
pred = ensemble.predict(test)



stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%



# 2. Generate a random grid of models and stack them together


In [10]:

# Specify GBM hyperparameters for the grid
hyper_params = {"learn_rate": [0.01, 0.03],
                "max_depth": [3, 4, 5, 6, 9],
                "sample_rate": [0.7, 0.8, 0.9, 1.0],
                "col_sample_rate": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}
search_criteria = {"strategy": "RandomDiscrete", "max_models": 3, "seed": 1}


In [11]:

# Train the grid
grid = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees=10,
                                                        seed=1,
                                                        nfolds=nfolds,
                                                        fold_assignment="Modulo",
                                                        keep_cross_validation_predictions=True),
                     hyper_params=hyper_params,
                     search_criteria=search_criteria,
                     grid_id="gbm_grid_binomial")
grid.train(x=x, y=y, training_frame=train)


gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
     col_sample_rate  learn_rate  max_depth  sample_rate  \
0                0.5        0.03        9.0          0.7   
1                0.8        0.01        9.0          0.9   
2                0.6        0.01        5.0          0.8   

                   model_ids   logloss  
0  gbm_grid_binomial_model_3  0.645560  
1  gbm_grid_binomial_model_2  0.672572  
2  gbm_grid_binomial_model_1  0.677297  


In [12]:

# Train a stacked ensemble using the GBM grid
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_gbm_grid_binomial",
                                       base_models=grid.model_ids)
ensemble.train(x=x, y=y, training_frame=train)


stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  my_ensemble_gbm_grid_binomial

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.14113983824822104
RMSE: 0.37568582385847493
LogLoss: 0.45079877068917285
Null degrees of freedom: 9999
Residual degrees of freedom: 9996
Null deviance: 13828.11338742428
Residual deviance: 9015.975413783457
AIC: 9023.975413783457
AUC: 0.9002029464566155
AUCPR: 0.9025750235806597
Gini: 0.8004058929132309

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4384824250060673: 


,,0,1,Error,Rate
0,0,3429.0,1276.0,0.2712,(1276.0/4705.0)
1,1,544.0,4751.0,0.1027,(544.0/5295.0)
2,Total,3973.0,6027.0,0.182,(1820.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.438482,0.839251,234.0
1,max f2,0.339351,0.902893,282.0
2,max f0point5,0.567829,0.836535,175.0
3,max accuracy,0.481957,0.820900,215.0
4,max precision,0.906863,1.000000,0.0
5,max recall,0.160242,1.000000,370.0
6,max specificity,0.906863,1.000000,0.0
7,max absolute_mcc,0.481957,0.640494,215.0
8,max min_per_class_accuracy,0.522783,0.819129,197.0
9,max mean_per_class_accuracy,0.522783,0.819196,197.0



Gains/Lift Table: Avg response rate: 52.95 %, avg score: 53.05 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.887677,1.888574,1.888574,1.000,0.893941,1.000000,0.893941,0.018886,0.018886,88.857413,88.857413,0.018886
1,2,0.02,0.881534,1.888574,1.888574,1.000,0.884523,1.000000,0.889232,0.018886,0.037771,88.857413,88.857413,0.037771
2,3,0.03,0.876378,1.888574,1.888574,1.000,0.879164,1.000000,0.885876,0.018886,0.056657,88.857413,88.857413,0.056657
3,4,0.04,0.872749,1.831917,1.874410,0.970,0.874807,0.992500,0.883109,0.018319,0.074976,83.191690,87.440982,0.074339
4,5,0.05,0.868839,1.831917,1.865911,0.970,0.871064,0.988000,0.880700,0.018319,0.093296,83.191690,86.591124,0.092020
5,6,0.10,0.844582,1.813031,1.839471,0.960,0.857183,0.974000,0.868941,0.090652,0.183947,81.303116,83.947120,0.178421
6,7,0.15,0.817155,1.767705,1.815549,0.936,0.831203,0.961333,0.856362,0.088385,0.272332,76.770538,81.554926,0.260005
7,8,0.20,0.784454,1.726157,1.793201,0.914,0.800805,0.949500,0.842473,0.086308,0.358640,72.615675,79.320113,0.337174
8,9,0.30,0.709327,1.631728,1.739377,0.864,0.748240,0.921000,0.811062,0.163173,0.521813,63.172805,73.937677,0.471441
9,10,0.40,0.630644,1.478754,1.674221,0.783,0.670275,0.886500,0.775865,0.147875,0.669688,47.875354,67.422096,0.573195


In [13]:

# Eval ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)


In [14]:

# Compare to base learner performance on the test set
baselearner_best_auc_test = max([h2o.get_model(model).model_performance(test_data=test).auc() for model in grid.model_ids])
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))


Best Base-learner Test AUC:  0.7678180918710861
Ensemble Test AUC:  0.773748647594226


In [15]:

# Generate predictions on a test set (if neccessary)
pred = ensemble.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [16]:
pred

predict,p0,p1
0,0.732173,0.267827
1,0.590758,0.409242
1,0.488445,0.511555
1,0.23445,0.76555
1,0.492359,0.507641
1,0.432636,0.567364
1,0.318592,0.681408
0,0.710543,0.289457
1,0.540632,0.459368
1,0.455489,0.544511
